# Assignment 7

### Brett Foster

##### DSC650-T303

In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import hashlib
import pygeohash as pgh
from pathlib import Path
import pyarrow as pa
from pyarrow.json import read_json
import pyarrow.parquet as pq

## 7a.

#### Specifies file paths

In [108]:
current_dir = Path(os.getcwd()).absolute()
data_dir = current_dir.joinpath('../assignment03/results')
df_file = data_dir.joinpath('routes.parquet')
results_dir = current_dir.joinpath('results')
results_dir.mkdir(exist_ok=True, parents=True)
kv_dir = results_dir.joinpath('kv')
kv_dir.mkdir(exist_ok=True, parents=True)

In [99]:
results_dir

WindowsPath('C:/datascience/dsc-650/repository/dsc650/dsc650/assignments/assignment07/results')

#### Loads the data from the routes.parquet file

In [10]:
df = pd.read_parquet(df_file)

In [11]:
df

,airline,src_airport,dst_airport,codeshare,equipment
0,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2965.0, 'name': 'Sochi Internat...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2]
1,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2]
2,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2962.0, 'name': 'Mineralnyye Vo...",False,[CR2]
3,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2]
4,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 4078.0, 'name': 'Tolmachevo Air...",False,[CR2]
...,...,...,...,...,...
67658,"{'airline_id': 4178, 'name': 'Regional Express...","{'airport_id': 6334.0, 'name': 'Whyalla Airpor...","{'airport_id': 3341.0, 'name': 'Adelaide Inter...",False,[SF3]
67659,"{'airline_id': 19016, 'name': 'Apache Air', 'a...","{'airport_id': 4029.0, 'name': 'Domodedovo Int...","{'airport_id': 2912.0, 'name': 'Manas Internat...",False,[734]
67660,"{'airline_id': 19016, 'name': 'Apache Air', 'a...","{'airport_id': 2912.0, 'name': 'Manas Internat...","{'airport_id': 4029.0, 'name': 'Domodedovo Int...",False,[734]
67661,"{'airline_id': 19016, 'name': 'Apache Air', 'a...","{'airport_id': 2912.0, 'name': 'Manas Internat...","{'airport_id': 2913.0, 'name': 'Osh Airport', ...",False,[734]


In [12]:
partitions = (
        ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
        ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
        ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
        ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

In [ ]:
airline src_airport dst_airport (iata)

In [38]:
df.iloc[100]['airline']

{'airline_id': 470,
 'name': 'Air Burkina',
 'alias': 'ANA All Nippon Airways',
 'iata': '2J',
 'icao': 'VBW',
 'callsign': 'BURKINA',
 'country': 'Burkina Faso',
 'active': True}

In [44]:
df['dst_airport'][100]

{'airport_id': 1084.0,
 'name': 'Léopold Sédar Senghor International Airport',
 'city': 'Dakar',
 'country': 'Senegal',
 'iata': 'DKR',
 'icao': 'GOOY',
 'latitude': 14.739700317382812,
 'longitude': -17.49020004272461,
 'altitude': 85.0,
 'timezone': 0.0,
 'dst': 'N',
 'tz_id': 'Africa/Dakar',
 'type': 'airport',
 'source': 'OurAirports'}

In [47]:
keys = []

for i in range(len(df)):
    try:
        src = df['src_airport'][i]['iata']
        dst = df['dst_airport'][i]['iata']
        air = df['airline'][i]['iata']
        res = (src + dst + air).upper()
        keys.append(res)
    except:
        keys.append(pd.NaT)

In [80]:
def keying(row):
    try:
        src = row['src_airport']['iata']
        dst = row['dst_airport']['iata']
        air = row['airline']['iata']
        res = (src + dst + air).upper()
        return(res)
    except:
        return(pd.NaT)

In [85]:
df['key'] = df.apply(lambda row: keying(row), axis=1)

In [87]:
df['key']

0        AERKZN2B
1        ASFKZN2B
2        ASFMRV2B
3        CEKKZN2B
4        CEKOVB2B
           ...   
67658    WYAADLZL
67659    DMEFRUZM
67660    FRUDMEZM
67661    FRUOSSZM
67662    OSSFRUZM
Name: key, Length: 67663, dtype: object

In [91]:
df['key'].isnull().sum()

892

In [94]:
df = df.dropna(subset=['key'])

In [115]:
def kv_keying(row):
    k = row['key'][0]
    if k in ['A', 'B', 'M', 'N', 'U', 'V']:
        return(k)
    elif k in ['C', 'D']:
        return('C-D')
    elif k in ['E', 'F']:
        return('E-F')
    elif k in ['G', 'H']:
        return('G-F')
    elif k in ['I', 'J']:
        return('I-J')
    elif k in ['K', 'L']:
        return('K-L')
    elif k in ['O', 'P']:
        return('O-P')
    elif k in ['Q', 'R']:
        return('Q-R')
    elif k in ['S', 'T']:
        return('S-T')
    elif k in ['W', 'X']:
        return('W-X')
    else:
        return('Y-X')

In [116]:
df['kv_key'] = df.apply(lambda row: kv_keying(row), axis=1)

<ipython-input-116-65f5d7b60829>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kv_key'] = df.apply(lambda row: kv_keying(row), axis=1)


In [117]:
df['kv_key']

0          A
1          A
2          A
3        C-D
4        C-D
        ... 
67658    W-X
67659    C-D
67660    E-F
67661    E-F
67662    O-P
Name: kv_key, Length: 66771, dtype: object

In [118]:
df.to_parquet(kv_dir, partition_cols=['kv_key'])

## 7b.

In [125]:
hash_dir = results_dir.joinpath('hash')
hash_dir.mkdir(exist_ok=True, parents=True)

In [120]:
def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return(m.hexdigest())

In [128]:
df['hashed'] = df['key'].apply(lambda key: hash_key(key))

<ipython-input-128-9fa354865aa9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hashed'] = df['key'].apply(lambda key: hash_key(key))


In [129]:
df['hashed'].head(20)

0     652cdec02010381f175efe499e070c8cbaac1522bac59a...
1     9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...
2     161143856af25bd4475f62c80c19f68936a139f653c1d3...
3     39aa99e6ae2757341bede9584473906ef1089e30820c90...
4     143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...
5     e4ec7b234cd26c4afd736cd49d1d02e4ec5f294f14533a...
6     30114a9dc60716adbadf6c54124a899a66eea47335fdae...
8     f763f8f658c0fb8dcedcf3b2ccbff0ac4e860cb0b96ef9...
9     ac5c008dc056e24302f24c6b88b98f1cbe2044d8061699...
10    dcbe3cf145ac9bd3ea56562a1db1ffdc40b8f2df3f3d84...
11    499c88c21f634b9451cef6ac444363e8b705f0bde459eb...
12    43c9d83e450469a16eba03dcb36531315a6a42d0860d6f...
13    ab62e10ee3376ac964b8a97a41a00cb18df0d7309f1dfe...
14    60b9791fe62049be32ac956c72b5883b2640d30baa2b41...
15    e375d99f34f301d2faf031ac9a17f9730e91c485971a72...
16    a55f0ed0b5f26db45f7f79b918484a78a33765446743c7...
17    901fb634e21acaa4e449bc820f094de3f016231e134434...
18    720a45cdc5864821d81b042e4e39764fa2ed56faa6

In [130]:
def hk(key):
    return(key[0].title())

In [131]:
df['hash_key'] = df['hashed'].apply(lambda key: hk(key))

<ipython-input-131-ef75ad404273>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hash_key'] = df['hashed'].apply(lambda key: hk(key))


In [133]:
df['hash_key'].head(10)

0     6
1     9
2     1
3     3
4     1
5     E
6     3
8     F
9     A
10    D
Name: hash_key, dtype: object

In [134]:
df.to_parquet(hash_dir, partition_cols=['hash_key'])

## 7c.

In [170]:
geo_dir = results_dir.joinpath('geo')
geo_dir.mkdir(exist_ok=True, parents=True)

In [171]:
west_code = pgh.encode(45.5945645, -121.1786823, precision=6)
central_code = pgh.encode(41.1544433, -96.0422378, precision=6)
east_code = pgh.encode(39.08344, -77.6497145, precision=6)

In [172]:
print(df['src_airport'][0]['latitude'])
print(df['src_airport'][0]['longitude'])

43.449902
39.9566


In [173]:
def geograph(row):
    lat = row['src_airport']['latitude']
    lon = row['src_airport']['longitude']
    geog = pgh.encode(lat, lon, precision=6)
    return(geog)

In [174]:
df['geohash'] = df.apply(lambda row: geograph(row), axis=1)

<ipython-input-174-af39a1d41558>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geohash'] = df.apply(lambda row: geograph(row), axis=1)


In [175]:
def geohashing_key(key):
    dist = {}
    west = pgh.geohash_approximate_distance(key, west_code)
    central = pgh.geohash_approximate_distance(key, central_code)
    east = pgh.geohash_approximate_distance(key, east_code)
    dist['west'] = west
    dist['central'] = central
    dist['east'] = east
    return(min(dist, key=dist.get))

In [176]:
df['geo_key'] = df['geohash'].apply(lambda row: geohashing_key(row))

<ipython-input-176-6febf3327c40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geo_key'] = df['geohash'].apply(lambda row: geohashing_key(row))


In [177]:
df['geo_key'].value_counts()

west       51311
east        9980
central     5480
Name: geo_key, dtype: int64

In [178]:
df.to_parquet(geo_dir, partition_cols=['geo_key'])

## 7d.

In [233]:
def balance_partitions(keys, num_partitions):
    partitions = []
    split_list = np.array_split(keys, num_partitions)
    for l in split_list:
        partitions.append(sorted(l))
    return(partitions)

In [234]:
listing1 = [5, 10, 25, 2, 6, 80, 4, 7, 40, 21, 18, 36, 57, 22, 200, 140, 17, 89, 81, 76]

In [236]:
balance_partitions(listing1, 6)

[[2, 5, 10, 25],
 [4, 6, 7, 80],
 [18, 21, 40],
 [22, 36, 57],
 [17, 140, 200],
 [76, 81, 89]]

In [237]:
balance_partitions(listing1, 2)

[[2, 4, 5, 6, 7, 10, 21, 25, 40, 80],
 [17, 18, 22, 36, 57, 76, 81, 89, 140, 200]]

In [238]:
balance_partitions(listing1, 8)

[[5, 10, 25],
 [2, 6, 80],
 [4, 7, 40],
 [18, 21, 36],
 [22, 57],
 [140, 200],
 [17, 89],
 [76, 81]]

In [239]:
balance_partitions(listing1, 4)

[[2, 5, 6, 10, 25],
 [4, 7, 21, 40, 80],
 [18, 22, 36, 57, 200],
 [17, 76, 81, 89, 140]]